In [5]:
import sqlite3

db = sqlite3.connect("my_db.sqlite") # This creates the files if doesn't exist

In [6]:
# Open file and run script
with open("tables.sql", "r") as file:
    query = file.read()
    db.executescript(query)

# Show the list of tables
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

OperationalError: unknown column "created_by" in foreign key definition

In [ ]:
# Open file and run script
with open("inserts.sql", "r") as file:
    query = file.read()
    db.executescript(query)